In [12]:
import pandas as pd
import unidecode

# Leer el archivo CSV
df = pd.read_csv('INVENTOR6724.csv')

# Seleccionar las columnas necesarias
df['NOMBRE DEL INVESTIGADOR PATENTE'] = df['inventor_last_name'] + ',' + df['inventor_first_name']

# Seleccionar y formatear nombres
df['NOMBRE DEL INVESTIGADOR PATENTE'] = (
    df['inventor_last_name'] + ',' + df['inventor_first_name']
    ).str.replace(r'[-]', ' ', regex=True).str.replace(r'[.]', '', regex=True).apply(unidecode.unidecode).str.upper()

# Ordenar en orden alfabético
df = df.sort_values(by='NOMBRE DEL INVESTIGADOR PATENTE').drop_duplicates(subset=['NOMBRE DEL INVESTIGADOR PATENTE'])

# Guardar en un nuevo archivo CSV
df[['NOMBRE DEL INVESTIGADOR PATENTE']].to_csv('Nombres_Patentes.csv', index=False)

In [13]:
# Leer el archivo Nombres_Limpios_Final
df_nombres_limpios = pd.read_csv('Nombres_Limpios_Final.csv')

# Buscar coincidencias
df_match = df[df['NOMBRE DEL INVESTIGADOR PATENTE'].isin(df_nombres_limpios['NOMBRE DEL INVESTIGADOR'])]

# Guardar las coincidencias en un nuevo archivo CSV
df_match[['NOMBRE DEL INVESTIGADOR PATENTE']].rename(columns={'NOMBRE DEL INVESTIGADOR PATENTE': 'NOMBRE DEL INVESTIGADOR MATCH'}).to_csv('Match.csv', index=False)